In [1]:
# Webscrape
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import requests
import pandas as pd
import datetime
import time
import json
import openpyxl

from web3.auto import Web3

# Quicknode wss endpoint: 
quicknode_API = "wss://thrilling-chaotic-flower.discover.quiknode.pro/4ac94983b963b326209e30c8c3c091fd4f61ff91/"
# Alchemy wss endpoint: 
alchemy_API = "wss://eth-mainnet.g.alchemy.com/v2/5Ffn88xmEVCEuD9sU_KfN3Grg-NrI_s9"
quicknode_node=Web3.WebsocketProvider(alchemy_API)
quicknode=Web3(quicknode_node)

# UniSwap

In [2]:
# Scrapping and utility functions

def get_pool_elements(driver):
    """Returns list of tuple [("address","description")]
    E.g. [('0x5777d92f208679db4b9778590fa3cab3ac9e2168', 'DAI/USDC 0.01%'), ...]"""
    
    # Get list of parent nodes 
    elements=driver.find_elements(By.CSS_SELECTOR, "a.sc-fYiAbW.dSQPJH")
    return elements

def uniswap_next_page(driver):
    """Flips to next page on uniswap pool page"""
    driver.find_elements(By.CSS_SELECTOR,'.sc-GMQeP.hXYrmI')[-1].click()
    return

# If removal affects nothing, remove this function
# def get_yester_date():
#     """Checks for current date(SGT), returns the previous date 9AM SGT in unix"""
#     yesterday = datetime.datetime.combine(datetime.date.today()-datetime.timedelta(days = 1), datetime.time(9,0,0))
#     return int(datetime.datetime.timestamp(yesterday))

In [3]:
# Get contract ABI
def get_ABI(contract_address):
    """Returns contractABI"""
    
    url="https://api.etherscan.io/api"
    params={
        "module":"contract",
        "action":"getabi",
        "address": contract_address, # Proxy address. Most/All uniswap pools uses same ABI
        "apikey":"NAA8H1TU9XKASRR6WXV263RZRHCH51TBWJ"
    }
    response=requests.get(url,params=params)
    return response.json()["result"]

# Since there are only 2 types of ABI, contractABI and ERC20ABI. Store them into variable instead of re-computing
pool_ABI=get_ABI("0x8f8ef111b67c04eb1641f5ff19ee54cda062f163") # Proxy pool contract
token_ABI = get_ABI("0xa2327a938febf5fec13bacfb16ae10ecbc4cbdcf")

# token_ABI=get_ABI("0x6B175474E89094C44Da98b954EedeAC495271d0F") # Proxy token contract

In [4]:
# Initialise contract
def initialise(cid, abi):
    contract=quicknode.eth.contract(address=quicknode.toChecksumAddress(cid),abi=abi)
    return contract

def query_functions(abi, cid, wid, functions, bid=None, uid = None ):
    # Runs the web3 functions specified in the parameters
    contract = initialise(cid, abi)
    results = []
    for function in functions: 
        if not isinstance(bid, int): 
            query = f"contract.functions.{function}.call()"
        else: 
            query = f"contract.functions.{function}.call(block_identifier = {bid})"
        
        result = eval(query)
        results.append(result)
    return results

In [5]:
# Utility functions
def get_yester_date(today: datetime.date):
    """Returns the datetime for yesterday, and if today is a Monday, return Friday datetime.date"""
    if today.weekday() == 0:
        delta = 3
    else:
        delta = 1

    yesterday = today - datetime.timedelta(days = delta)
    return yesterday

def get_bid(unix_time):
    """"Takes unix time and return block height"""
    response=requests.get(f"https://coins.llama.fi/block/ethereum/{unix_time}")
    return response.json()["height"]

## Webscrapping
Get the addresses and pool_desc from uniswap

In [3]:
# Launches interactive chrome
options = Options()
options.add_argument("--headless")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)
driver.set_window_size(1024, 600)
driver.maximize_window()

driver.get("https://info.uniswap.org/#/pools")
driver.maximize_window()

WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x55c8ea1a6133 <unknown>
#1 0x55c8e9eda966 <unknown>
#2 0x55c8e9f01b57 <unknown>
#3 0x55c8e9f0007b <unknown>
#4 0x55c8e9f3f9ad <unknown>
#5 0x55c8e9f3f18f <unknown>
#6 0x55c8e9f369a3 <unknown>
#7 0x55c8e9f0b46a <unknown>
#8 0x55c8e9f0c55e <unknown>
#9 0x55c8ea165cae <unknown>
#10 0x55c8ea1698fe <unknown>
#11 0x55c8ea172f20 <unknown>
#12 0x55c8ea16a923 <unknown>
#13 0x55c8ea13dc0e <unknown>
#14 0x55c8ea18db08 <unknown>
#15 0x55c8ea18dc97 <unknown>
#16 0x55c8ea19e113 <unknown>
#17 0x7f908a8d1609 start_thread


In [ ]:
pages_to_flip=5 # Go through 3 pages and get all elements

addresses=[] # Stores addresses
pools_desc=[] # Stores pool description
for page in range(pages_to_flip):
    driver.implicitly_wait(10) # Wait 10s for page to load
    elements=get_pool_elements(driver) # Get parent elements for the different pools
    
    for i in elements:
        addresses.append(i.get_attribute("href")[-42:])
        pools_desc.append(i.find_element(By.CSS_SELECTOR,".sc-chPdSV.goKJOd.css-1j9mh98").text + " " + 
                             i.find_element(By.CSS_SELECTOR,".sc-jKJlTe.sc-gipzik.sc-fAjcbJ.dsySXo").get_attribute("innerHTML"))
        
    uniswap_next_page(driver)# Flips to next page

## Query blockchain
Get the qty of tokens in a pool for today and yesterday.

In [ ]:
# today=datetime.date(year=2023,month=2,day=18) # This line is for backtesting
today = datetime.date.today()
today = datetime.datetime.combine(today, datetime.time(1,0,0)) # SGT is 8hours ahead, so 1AM UTC is 9AM SGT
yesterday = get_yester_date(today)

today_unix = int(datetime.datetime.timestamp(today))
yester_unix = int(datetime.datetime.timestamp(yesterday))

In [ ]:
token_amount_dict={}
for pool_address in addresses:
    # Input needed to lookup blockchain
    cid = quicknode.toChecksumAddress(pool_address.lower())
    wid = quicknode.toChecksumAddress(pool_address.lower())
    uid = -1
    yester_bid = get_bid(yester_unix)
    today_bid=get_bid(today_unix)
    functions = ["token0()","token1()"]
    
    # Get the tokens' CID
    tokens_cid=query_functions(abi=pool_ABI,cid=cid,wid=wid,functions=functions,uid=uid)
    assert bool(tokens_cid), "tokens_cid is empty" # Ensures tokens_cid is not empty list
    
    # Add quantity of tokens into dict where key is (wallet_id, token_cid)
    functions=["balanceOf(wid)","decimals()"]
    for token_cid in tokens_cid:
        try: 
            # Yesterday's amount
            yester_amount, decimals=query_functions(abi=token_ABI, cid=token_cid,wid=wid,functions=functions,bid=yester_bid,uid=uid)
            yester_amount = yester_amount / 10**decimals
    
            # Today's amount
            today_amount, decimals=query_functions(abi=token_ABI, cid=token_cid,wid=wid,functions=functions,bid=today_bid,uid=uid)
            today_amount = today_amount / 10**decimals
            
            # Add to dictionary
            token_amount_dict[(wid.lower(),token_cid.lower())] = [yester_amount,today_amount]
        except:
            print(f"Pool {wid} is skipped")

In [ ]:
df = pd.DataFrame(token_amount_dict.values(),index = token_amount_dict.keys(), columns = ["yester_qty","today_qty"])
df.index.set_names(["wallet_id","token_cid"],inplace=True)

# Curve

## Webscrapping

In [ ]:
driver.get("https://curve.fi/#/ethereum/pools")
driver.implicitly_wait(10)

# Sort table by descending TVL
sortTVL_button=driver.find_elements(By.TAG_NAME,"th")[4].find_element(By.TAG_NAME, "button")
sortTVL_button.click()

actions=ActionChains(driver)

pool_rows = driver.find_elements(By.CSS_SELECTOR, ".sc-8rvjwc-2.bYiYAe")
crv_dict = {}
crv_hash = {} # pool_address: pool_desc
for i in range(40):
    current_row = pool_rows [i]
    
    # Pool
    pool = current_row.find_element(By.CSS_SELECTOR,".sc-14f2b9o-0.fzBqXX")
    if not pool.is_displayed():
        print("pool scrolling...")
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", pool)
        wait = WebDriverWait(driver, 10)
        wait.until(EC.visibility_of(pool))
    
    actions.move_to_element(pool).perform()
    pool_address = driver.find_element(By.CSS_SELECTOR,".sc-1n9dsqf-3.xanQo").text

    # Token address and pool description
    tokens = current_row.find_elements(By.TAG_NAME,"img")
    num_of_tokens = len(tokens)
    pool_desc = current_row.find_element(By.CSS_SELECTOR,".sc-1ddj6ya-0.sc-1l6ho57-2.bpcBii.hKRpvQ").text
    tokens = current_row.find_elements(By.CSS_SELECTOR, ".sc-14f2b9o-0.jPTkUD.sc-1l6ho57-0.dLcGVs")
    for token in tokens:
        # LEFT OFF HERE
        # PROBLEM - Trying to get the token_address through mouse hover instead of DOM
        # DOM would have some image having the name of crypto-it-8366de0...
        actions.move_to_element(token).perform()
        token_address = driver.find_element(By.CSS_SELECTOR,".sc-1n9dsqf-3.xanQo").text
        crv_dict[(pool_address, token_address)] = None
        crv_hash[pool_address] = pool_desc

driver.close()

[<selenium.webdriver.remote.webelement.WebElement (session="67509549332d804e4e8ca6148945a2ce", element="c9107d97-244f-4bc9-8e5e-e344b5e734ca")>, <selenium.webdriver.remote.webelement.WebElement (session="67509549332d804e4e8ca6148945a2ce", element="d0fa7766-a231-480e-9a5e-8029637f21ce")>, <selenium.webdriver.remote.webelement.WebElement (session="67509549332d804e4e8ca6148945a2ce", element="ebf01429-36c6-4624-8c9b-2e13a17beb4c")>, <selenium.webdriver.remote.webelement.WebElement (session="67509549332d804e4e8ca6148945a2ce", element="69e2e5ca-ad7e-44f7-babe-404fa7115e7f")>, <selenium.webdriver.remote.webelement.WebElement (session="67509549332d804e4e8ca6148945a2ce", element="337ae37c-a50d-42d3-97b7-b6a0492e0891")>, <selenium.webdriver.remote.webelement.WebElement (session="67509549332d804e4e8ca6148945a2ce", element="28c6426f-ad6c-451e-b188-2e00805463fe")>, <selenium.webdriver.remote.webelement.WebElement (session="67509549332d804e4e8ca6148945a2ce", element="a8970869-0d78-44fd-b9e1-3f949321

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".sc-14f2b9o-0.fzBqXX"}
  (Session info: headless chrome=90.0.4430.212)
Stacktrace:
#0 0x56043f3c37f9 <unknown>
#1 0x56043f3633b3 <unknown>
#2 0x56043f0ab016 <unknown>
#3 0x56043f0df81e <unknown>
#4 0x56043f0d6e67 <unknown>
#5 0x56043f102ded <unknown>
#6 0x56043f0d6d7b <unknown>
#7 0x56043f102fce <unknown>
#8 0x56043f1139e1 <unknown>
#9 0x56043f102c93 <unknown>
#10 0x56043f0d4ce4 <unknown>
#11 0x56043f0d64d2 <unknown>
#12 0x56043f38f542 <unknown>
#13 0x56043f39ece7 <unknown>
#14 0x56043f39e9e4 <unknown>
#15 0x56043f3a313a <unknown>
#16 0x56043f39f5b9 <unknown>
#17 0x56043f384e00 <unknown>
#18 0x56043f3b65d2 <unknown>
#19 0x56043f3b6778 <unknown>
#20 0x56043f3cea1f <unknown>
#21 0x7ffacc268fa3 start_thread
#22 0x7ffacb4bf06f clone


## Query Blockchain

In [ ]:
for pool_address, token_address in crv_dict.keys():
    # Input needed to lookup blockchain
    cid = quicknode.toChecksumAddress(token_address.lower())
    wid = quicknode.toChecksumAddress(pool_address.lower())
    uid = -1
    yester_bid = get_bid(yester_unix)
    today_bid=get_bid(today_unix)

    # Add quantity of tokens into dict where key is (wallet_id, token_cid)
    functions=["balanceOf(wid)","decimals()"]
    for token_cid in tokens_cid:
        try: 
            # Yesterday's amount
            yester_amount, decimals=query_functions(abi=token_ABI, cid=token_cid,wid=wid,functions=functions,bid=yester_bid,uid=uid)
            yester_amount = yester_amount / 10**decimals
    
            # Today's amount
            today_amount, decimals=query_functions(abi=token_ABI, cid=token_cid,wid=wid,functions=functions,bid=today_bid,uid=uid)
            today_amount = today_amount / 10**decimals
            
            # Add to dictionary
            token_amount_dict[(wid.lower(),token_cid.lower())] = [yester_amount,today_amount]
        except:
            print(f"Pool {wid} is skipped")

In [ ]:
token_amount_dict

## Price 

In [ ]:
# Get price_df from coingecko
# Methodology: Use token_cid to look up coingecko's historical price
def check_for_closest_timestamp(ideal_timestamp: int, price_dict):
    """Compares the ideal 9AM timestamp to the closest timestamp, returns (timestamp,price)"""
    ideal_timestamp*=1000 # Converts to millisecond timestamp
    
    bestfit_timestamp,bestfit_price=0,0
    bestfit_difference=1000000000000 # Random number
    for timestamp, price in price_dict:
        difference=abs(ideal_timestamp-timestamp)
        
        if difference<bestfit_difference:
            bestfit_difference = difference # Comparison basis
            bestfit_timestamp,bestfit_price = timestamp,price # Thing i actually want
        elif difference>bestfit_difference:
            # Once the search starts getting further away
            break

    # Return the timestamp in seconds
    return bestfit_timestamp/1000,bestfit_price

def price_from_cid(contract_addresses: list):
    """Return a dictionary with structure {(token_cid0,time0):price0, (token_cid0,time1):price1...}"""
    THROTTLE_RATE=10 # 10s per API request
    
    token_time_price_dict={}
    for contract_address in contract_addresses:
        coingecko_url=f"https://api.coingecko.com/api/v3/coins/eth/contract/{contract_address}/market_chart/?vs_currency=usd&days=5"
        try: 
            price_dict=requests.get(coingecko_url).json()["prices"]

            # Yesterdays data
            timestamp,price=check_for_closest_timestamp(yester_unix,price_dict)
            token_time_price_dict[(contract_address,timestamp)] = price
            
            # Todays data
            timestamp,price=check_for_closest_timestamp(today_unix,price_dict)
            token_time_price_dict[(contract_address,timestamp)] = price
            
        except Exception as e:
            print(coingecko_url)
            print(f"Token CID cant be found: {contract_address}")
            
#         Throttle before moving onto next request
        time.sleep(THROTTLE_RATE)
      
    return token_time_price_dict

In [ ]:
contract_addresses=df.index.levels[1].unique()
my_dict=price_from_cid(contract_addresses)

In [ ]:
price_df=pd.DataFrame(my_dict.values(),index=my_dict.keys(),columns=["price"])
price_df.index.set_names(["token_cid","timestamp"],inplace=True) # Setting up names for the index

## Computation
Calculates the change in TVL for each pool using the price_df and the TVL from today and yesterday

In [ ]:
yester_TVL=[]
today_TVL=[]
for idx, df_select in df.groupby(level=[0,1]):
    token_cid = idx[1]
    if token_cid in price_df.index.get_level_values("token_cid"):
        try:
            df.loc[idx,"yester_TVL"] = price_df.loc[token_cid].iloc[0]["price"] * df_select.loc[idx,"yester_qty"]
            df.loc[idx,"today_TVL"] = price_df.loc[token_cid].iloc[1]["price"] * df_select.loc[idx,"today_qty"]
        except IndexError as e:
            print(f"Index error, {token_cid} not created yet")

In [ ]:
# Removes wallet with at least one token with missing price from coingecko
def drop_index(group):
    if group.isnull().values.any():
        return None
    else:
        return group

# group by level 0 index and apply custom function to drop index if it contains NaN values
df = df.groupby(level=0,group_keys=False).apply(drop_index)

In [ ]:
idx_wallets = df.index.get_level_values(0)
process_df = pd.DataFrame(data={},index=idx_wallets)

# Process the data in process_df
process_df = df.groupby(level=0).agg({"yester_TVL": "sum", "today_TVL": "sum"})
process_df["TVL_change"] = process_df["today_TVL"] - process_df["yester_TVL"]
process_df["percent_change"] = process_df["TVL_change"]/process_df["yester_TVL"] * 100

In [ ]:
# Replace wallet_id with pool desc
for wallet_id, pool_desc in zip(addresses, pools_desc):
    process_df.rename({wallet_id:pool_desc},inplace=True)
process_df.sort_values(by=["today_TVL"],ascending=False, inplace=True)

In [ ]:
# Export data before filtering
process_df.to_excel(f"uniswap_scrapper.xlsx")

## Filtering dataframe
* Only use pools with >$50M
* Get top 3 pools for both net TVL change and percentage change

In [ ]:
filt_20M = process_df["yester_TVL"] >= 20000000

# Filter by percent_change and get top 3 winners/losers
topwinner_percentage_df = process_df[filt_20M].sort_values(by=["percent_change"],ascending=False)[:3]
toploser_percentage_df = process_df[filt_20M].sort_values(by=["percent_change"])[:3]

# Filter by TVL_change and get top 3 winners/losers
topwinner_abs_df = process_df[filt_20M].sort_values(by=["TVL_change"],ascending=False)[:3]
toploser_abs_df = process_df[filt_20M].sort_values(by=["TVL_change"])[:3]

# Drop unneeded columns
topwinner_percentage_df.drop(["yester_TVL","today_TVL","TVL_change"],axis=1,inplace=True)
toploser_percentage_df.drop(["yester_TVL","today_TVL","TVL_change"],axis=1,inplace=True)
topwinner_abs_df.drop(["yester_TVL","today_TVL","percent_change"],axis=1,inplace=True)
toploser_abs_df.drop(["yester_TVL","today_TVL","percent_change"],axis=1,inplace=True)

In [ ]:
# Filter top 3 winner/losers
with pd.ExcelWriter(f"daily_top3.xlsx") as writer:
    # Percentage
    topwinner_percentage_df.to_excel(writer, sheet_name="Sheet1",startcol=0, startrow=1)
    toploser_percentage_df.to_excel(writer, sheet_name="Sheet1",startcol=2, startrow=1)

    # Absolute
    topwinner_abs_df.to_excel(writer, sheet_name="Sheet1",startcol=5, startrow=1)
    toploser_abs_df.to_excel(writer, sheet_name="Sheet1",startcol=7, startrow=1)

# Merge and create cells: "Top Winners" and "Top Losers"
workbook = openpyxl.load_workbook(f'daily_top3.xlsx')
worksheet = workbook['Sheet1']

worksheet.merge_cells('A1:B1')
worksheet.merge_cells('C1:D1')
worksheet['A1'] = 'Top Performers'
worksheet['C1'] = 'Worst Performers'

worksheet.merge_cells('F1:G1')
worksheet.merge_cells('H1:I1')
worksheet['F1'] = 'Top Performers'
worksheet['H1'] = 'Worst Performers'

workbook.save(f'daily_top3.xlsx')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3aa94a54-c990-4933-b9b0-4e62960dfc0f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>